# buildFinalOutputFinal

In [ ]:
// Per frame: Two feet. By foot: (x y w h code xp yp d)
//  (x,y,w,h): foot rect                (left_step, right_step)
//  code:                               (in_objective1, in_objective2)
//      0: No step
//    1-9: Step to nearest objective
//  (xp,yp): Feet contact point         (left_foot, right_foot)
//  d: distance to nearest center       (odist1, odist2)

std::string ComputerVisionWeb::buildFinalOutputFinal(FeetTracker &ft, std::vector<MarkAndTime> sequence, int maxFrame) {
    //Get central stimulus central position
    cv::Point2f pcentral = ft.contourCentersScene[4];
    
    const int relevant_change = 10; //Number of centimeters for considering relevant change in position
    const int static_step = 15; //Number of frames for considering that step is not displacing
    
    int current_seq = 0, //index for starting current sequence
        current_center_exit1 = 0, current_center_exit2 = 0, //index for exiting center on current sequence for each foot
        next_seq = 0;  //index for starting next sequence
    cv::Point p_out1, p_out2;
    
    //Variables for stimuli sequence:
    uint n_objectives = sequence.size();
    int i, cur_objective, cur_frame;

    //Divide items in stimuli sequence data and calculate frames de despegue y llegada
    // Lista para almacenar los resultados
    std::vector<Section> sequences;
    bool first_stimulus = true;
    
    //Get intervals per objective:
    for (int j = 0; j < n_objectives; ++j) {
        Section cur_section;
        item cur_item;
        
        cur_objective = sequence[j].mark_correct; 
        cur_frame = sequence[j].frame;

#ifdef SHOW_DEBUG_TEXT
        std::cout << "Marking.\n\tCurrent stimulus: " << j << std::endl;
        std::cout << "\tCurrent stimulus objective: " << cur_objective << std::endl;
        std::cout << "\tCurrent stimulus index: " << cur_frame << std::endl;
#endif        
        //Booleans for marking errors (assume right first):
        bool step_center = true, step_objective = true, right_objective = true; 
        
        //Advance until both are near the 5 zone (assume that the player can be late):


        for (i = current_seq; i < maxFrame; ++i)
            if(ft.in_objective1[i] == 5 || ft.in_objective2[i] == 5)
                break;
#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tCurrent sequence start - prev cur_frame: " << current_seq << std::endl;
#endif        
        
        //Update start of current seq: if stimulus presentation is higher than presence in zone 5, start from stimulus presentation frame
        current_seq = (cur_frame >= i)? cur_frame : i;

#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tCurrent sequence start - after cur_frame: " << current_seq << std::endl;
#endif        

        
        //Search for center position exit, considered as the first step out of center zone:
        bool step_out_detected1 = false, step_out_detected2 = false;
        int sure_frame1, sure_frame2;

        //Search for left exit:
        for (i = current_seq; i < maxFrame; ++i) {
            if(ft.left_step[i] == 0) //Continue until a step is detected
                continue;
#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tCurrent frame - search left position exit: " << i << std::endl;
#endif        
            //Left foot steps out:
            if(ft.in_objective1[i] != 5) {
#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tExit frame left - position exit: " << i << std::endl;
        std::cout << "\tExit frame left - code found: " << ft.in_objective1[i] << std::endl;
#endif        
                current_center_exit1 = i;
                p_out1 = ft.left_foot[i];
                sure_frame1 = i;
                step_out_detected1 = true;
                break;
            }            
        }

        //Search for right exit:
        for (i = current_seq; i < maxFrame; ++i) {
            if(ft.right_step[i] == 0) //Continue until a step is detected
                continue;
#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tCurrent frame - search right position exit: " << i << std::endl;
#endif        
            //Right foot steps out:
            if(ft.in_objective2[i] != 5) {
#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tExit frame right - position exit: " << i << std::endl;
        std::cout << "\tExit frame right - code found: " << ft.in_objective2[i] << std::endl;
#endif        
                current_center_exit2 = i;
                p_out2 = ft.right_foot[i];
                sure_frame2 = i;
                step_out_detected2 = true;
                break;
            }            
        }

        
        
        if(step_out_detected1 || step_out_detected2) { //It shall be detected... if not maybe end of stimuli sequence or player skip some stimuli
            //Check which is the first leg going on the objective direction
            if(!step_out_detected1) { //Give a reference the second sure step if first not found
                p_out1 = p_out2;
                current_center_exit1 = current_center_exit2;
                sure_frame1 = sure_frame2;
            }
            if(!step_out_detected2) { //Give a reference the second sure step if first not found
                p_out2 = p_out1;
                current_center_exit2 = current_center_exit1;
                sure_frame2 = sure_frame1;
            }
            
            int lindex_1, lindex_2, rindex_1, rindex_2, il_found, ir_found, il_last = current_seq+1, ir_last = current_seq+1, il_initial, il_last_stepping, ir_initial, ir_last_stepping;
            bool stepping = false, first = true, pl_found = false, pr_found = false;
            cv::Point2f p1, p2, p_out_s = ft.imageToScene(p_out1), p_center = ft.contourCentersScene[4]; //Take central point as reference
            float d1, d2, d, 
                  d_center_obj = sqrt((p_out_s.x - p_center.x)*(p_out_s.x - p_center.x) + (p_out_s.y - p_center.y)*(p_out_s.y - p_center.y));
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tProcessing take-off left..." << std::endl;
                std::cout << "\tCenter (x,y): " << p_center.x << ", " << p_center.y << std::endl;
                std::cout << "\tSure out (x,y): " << p_out_s.x << ", " << p_out_s.y << std::endl;
#endif

            //Check backwards first relevant change in left step keeping going far sure feet and near center
            for (i = sure_frame1; i >= current_seq; --i) {
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tTake-off - left - frame: " << i << std::endl;
#endif                        
                if(first) { //Search for end of first stepping
                    if(!stepping && ft.left_step[i] == 1) { //A step
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - left - start stepping... " << std::endl;
#endif                        
                        stepping = true;
                        il_last = i;
                        il_initial = i;
                        lindex_1 = i;
                    } else if(stepping && ft.left_step[i] == 0) { //Is stepping, so check if it stops doing so
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - left - stop stepping... " << std::endl;
#endif                  
                        il_last_stepping = i-1;      
                        stepping = false;
                        first = false; //First index ready
                    }
                } else { //Search for first of following step
                    if(ft.left_step[i] == 1) { //First position of next step
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - left - left following found... " << std::endl;
#endif        
                        il_last = i;
                        lindex_2 = i;
                        //Significant displacement criterion
                        p1 = ft.imageToScene(ft.left_foot[lindex_1]);
                        p2 = ft.imageToScene(ft.left_foot[lindex_2]);
                        d  = sqrt((p2.x - p1.x)*(p2.x - p1.x) + (p2.y - p1.y)*(p2.y - p1.y));
                        //d1 = sqrt((p_out_s.x - p1.x)*(p_out_s.x - p1.x) + (p_out_s.y - p1.y)*(p_out_s.y - p1.y));//L2 norm
                        //d2 = sqrt((p_out_s.x - p2.x)*(p_out_s.x - p2.x) + (p_out_s.y - p2.y)*(p_out_s.y - p2.y));//L2 norm
                        d1 = magnitude(projectVector(cv::Point2f(p1.x - p_out_s.x, p1.y - p_out_s.y), cv::Point2f(p_center.x - p_out_s.x, p_center.y - p_out_s.y)));
                        d2 = magnitude(projectVector(cv::Point2f(p2.x - p_out_s.x, p2.y - p_out_s.y), cv::Point2f(p_center.x - p_out_s.x, p_center.y - p_out_s.y)));
                        
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tCenter (x,y): " << p_center.x << ", " << p_center.y << std::endl;
                        std::cout << "\tSure out (x,y): " << p_out_s.x << ", " << p_out_s.y << std::endl;
                        std::cout << "\tFoot 1 (x,y): " << p1.x << ", " << p1.y << " at index " << lindex_1 << std::endl;
                        std::cout << "\tFoot 2 (x,y): " << p2.x << ", " << p2.y << " at index " << lindex_2 << std::endl;                        
                        std::cout << "\tTake-off - left - distance between steps: " << d << std::endl;
                        std::cout << "\tTake-off - left - projected distance between 1st step and objective: " << d1 << std::endl;
                        std::cout << "\tTake-off - left - projected distance between 2nd step and objective: " << d2 << std::endl;
                        std::cout << "\tTake-off - left - distance between center and objective: " << d_center_obj << std::endl;
#endif
                        if(d1 < d2 && d > relevant_change && il_initial - il_last_stepping < static_step && d2 < d_center_obj && ft.odist1[i] != 0) { //While the step is approaching to the objective and not farther than center, keep searching...
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - left - keeps approaching objective... " << std::endl;
#endif
                            stepping = true;                            
                            first = true; //Consider as first again
                            lindex_1 = lindex_2;
                            il_initial = lindex_1;
                            continue;
                        }
                                
                        //If we reach here, it means that p1 is the take-off step
                        pl_found = true;
          
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - left - il initial:" << il_initial << std::endl;
                        std::cout << "\tTake-off - left - il last stepping:" << il_last_stepping << std::endl;
                        std::cout << "\tTake-off - left - stepping diff:" << il_initial - il_last_stepping << std::endl;
#endif                        
                        
                        if(il_initial - il_last_stepping >= static_step) { //Static step, so this is considered the take-off step
                            il_found = il_initial + 1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - left - static step: Found at " << il_found << "." << std::endl;
#endif
                            break;
                        }
                        
                        if(d <= relevant_change) { // If two little steps, assume second is the take_off
                            il_found = lindex_2+1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - left - irrelevant step distance: Found at " << il_found << "." << std::endl;
#endif        

                            break;
                        }
                        
                        if(d1 > d2) { //If p1 is farther than p2, it is the take-off step
                            il_found = il_initial + 1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - left - first step farther than second: Found at " << il_found << ". " << std::endl;
#endif
                        } else { //Else, p2 is.                            
                            il_found = lindex_2+1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - left - second step farther than first: Found at " << il_found << ". " << std::endl;
#endif
                        }
                        break;
                    }
                }
            }
            //If left not found, take last frame stepping as take-off:
            if(!pl_found) {
                il_found = il_last + 1;
                pl_found = true;
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tTake-off - left - not found so take last step: Found at " << il_found << ". " << std::endl;
#endif
            }

            //Now check first relevant change in right step
            stepping = false; 
            first = true;
            p_out_s = ft.imageToScene(p_out2);
            d_center_obj = sqrt((p_out_s.x - p_center.x)*(p_out_s.x - p_center.x) + (p_out_s.y - p_center.y)*(p_out_s.y - p_center.y));
#ifdef SHOW_DEBUG_TEXT
            std::cout << "\tProcessing take-off right..." << std::endl;
            std::cout << "\tCenter (x,y): " << p_center.x << ", " << p_center.y << std::endl;
            std::cout << "\tSure out (x,y): " << p_out_s.x << ", " << p_out_s.y << std::endl;
#endif
            for (i = sure_frame2; i >= current_seq; --i) {
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tTake-off - right - frame: " << i << std::endl;
#endif                        
                if(first) { //Search for end of first stepping
                    if(!stepping && ft.right_step[i] == 1) { //A step
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - right - start stepping... " << std::endl;
#endif                        
                        stepping = true;
                        ir_last = i;
                        ir_initial = i;
                        rindex_1 = i;
                    } else if(stepping && ft.right_step[i] == 0) { //Is stepping, so check if it stops doing so
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - right - stop stepping... " << std::endl;
#endif                  
                        ir_last_stepping = i+1;
                        stepping = false;
                        first = false; //First index ready
                    }
                } else { //Search for first of following step
                    if(ft.right_step[i] == 1) { //First position of next step
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - right - right following found... " << std::endl;
#endif        
                        ir_last = i;
                        rindex_2 = i;
                        //Significant displacement criterion
                        p1 = ft.imageToScene(ft.right_foot[rindex_1]);
                        p2 = ft.imageToScene(ft.right_foot[rindex_2]);
                        d  = sqrt((p2.x - p1.x)*(p2.x - p1.x) + (p2.y - p1.y)*(p2.y - p1.y));
//                        d1 = sqrt((p_out_s.x - p1.x)*(p_out_s.x - p1.x) + (p_out_s.y - p1.y)*(p_out_s.y - p1.y));//L2 norm
//                        d2 = sqrt((p_out_s.x - p2.x)*(p_out_s.x - p2.x) + (p_out_s.y - p2.y)*(p_out_s.y - p2.y));//L2 norm
                        d1 = magnitude(projectVector(cv::Point2f(p1.x - p_out_s.x, p1.y - p_out_s.y), cv::Point2f(p_center.x - p_out_s.x, p_center.y - p_out_s.y)));
                        d2 = magnitude(projectVector(cv::Point2f(p2.x - p_out_s.x, p2.y - p_out_s.y), cv::Point2f(p_center.x - p_out_s.x, p_center.y - p_out_s.y)));

#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tCenter (x,y): " << p_center.x << ", " << p_center.y << std::endl;
                        std::cout << "\tSure out (x,y): " << p_out_s.x << ", " << p_out_s.y << std::endl;
                        std::cout << "\tFoot 1 (x,y): " << p1.x << ", " << p1.y << " at index " << rindex_1 << std::endl;
                        std::cout << "\tFoot 2 (x,y): " << p2.x << ", " << p2.y << " at index " << rindex_2 << std::endl;   
                        std::cout << "\tTake-off - right - distance between steps: " << d << std::endl;
                        std::cout << "\tTake-off - right - projected distance between 1st step and objective: " << d1 << std::endl;
                        std::cout << "\tTake-off - right - projected distance between 2nd step and objective: " << d2 << std::endl;
                        std::cout << "\tTake-off - right - distance between center and objective: " << d_center_obj << std::endl;
#endif
                        if(d1 < d2 && d > relevant_change && ir_initial - ir_last_stepping < static_step && d2 < d_center_obj  && ft.odist2[i] != 0) { //While the step is approaching to the objective and not farther than center, keep searching...
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - right - keeps approaching objective... " << std::endl;
#endif
                            stepping = true;                            
                            first = true; //Consider as first again
                            rindex_1 = rindex_2;
                            ir_initial = rindex_1;
                            continue;
                        }
                                
                        //If we reach here, it means that p1 is the take-off step
                        pr_found = true;
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tTake-off - right - ir initial:" << ir_initial << std::endl;
                        std::cout << "\tTake-off - right - ir last stepping:" << ir_last_stepping << std::endl;
                        std::cout << "\tTake-off - right - stepping diff:" << ir_initial - ir_last_stepping << std::endl;
#endif        
                        
                        if(ir_initial - ir_last_stepping >= static_step) { //Static step, so this is considered the take-off step
                            ir_found = ir_initial + 1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - right - static step: Found at " << ir_found << "." << std::endl;
#endif        
                            break;
                        }
                        
                        if(d <= relevant_change) { // If two little steps, assume second is the take_off
                            ir_found = rindex_2+1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - right - irrelevant step distance: Found at " << ir_found << "." << std::endl;
#endif        
                            break;
                        }
                        
                        if(d1 > d2) { //If p1 is farther than p2, it is the take-off step
                            ir_found = ir_initial+1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - right - first step farther than second: Found at " << ir_found << "." << std::endl;
#endif
                        } else { //Else, p2 is.                            
                            ir_found = rindex_2+1;
#ifdef SHOW_DEBUG_TEXT
                            std::cout << "\tTake-off - right - second step farther than first: Found at " << ir_found << "." << std::endl;
#endif
                        }
                        break;
                    }
                }
            }
            //If left not found, take last frame stepping as take-off:
            if(!pr_found) {
                ir_found = ir_last + 1;
                pr_found = true;
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tTake-off - right - not found so take last step: Found at " << ir_found << ". " << std::endl;
#endif
            }
            
            //Set take-off frame:
            if(pl_found && pr_found) {
                if(il_found < ir_found) { //Take-off is from left
                    cur_item.code = 5;
                    cur_item.d_l = ft.odist1[il_found];
                    cur_item.d_r = ft.odist2[ir_found];
                    cur_item.step_l = true;
                    cur_item.step_r = false;
                    cur_item.frame = il_found;
                    cur_section.items.push_back(cur_item);
                    cur_section.takeoff_frame = il_found;
#ifdef SHOW_DEBUG_TEXT
                    std::cout << "\tTake-off: Both found - decided left" << std::endl;
#endif
                } else {
                    cur_item.code = 5;
                    cur_item.d_l = ft.odist1[il_found];
                    cur_item.d_r = ft.odist2[ir_found];
                    cur_item.step_l = false;
                    cur_item.step_r = true;
                    cur_item.frame = ir_found;                    
                    cur_section.items.push_back(cur_item);
                    cur_section.takeoff_frame = ir_found;
#ifdef SHOW_DEBUG_TEXT
                    std::cout << "\tTake-off: Both found - decided right" << std::endl;
#endif
                }
            } else if(pl_found) {
                cur_item.code = 5;
                cur_item.d_l = ft.odist1[il_found];
                cur_item.d_r = ft.odist2[il_last];
                cur_item.step_l = true;
                cur_item.step_r = false;
                cur_item.frame = il_found;
                cur_section.items.push_back(cur_item);
                cur_section.takeoff_frame = il_found;
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tTake-off: One found - left" << std::endl;
#endif
            } else if (pr_found) {
                cur_item.code = 5;
                cur_item.d_l = ft.odist1[il_found];
                cur_item.d_r = ft.odist2[ir_found];
                cur_item.step_l = false;
                cur_item.step_r = true;
                cur_item.frame = ir_found;                                    
                cur_section.items.push_back(cur_item);
                cur_section.takeoff_frame = ir_found;
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tTake-off: One found - right" << std::endl;
#endif
            } else { //if none, use max
                cur_item.code = 5;
                cur_item.d_l = ft.odist1[il_found];
                cur_item.d_r = ft.odist2[ir_found];
                cur_item.step_l = il_found;
                cur_item.step_r = ir_found;
                cur_item.frame = il_found<ir_found? il_found : ir_found;                                    
                cur_section.items.push_back(cur_item);
                cur_section.takeoff_frame = il_found<ir_found? il_found : ir_found;
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tTake-off: None found" << std::endl;
#endif
            }
#ifdef SHOW_DEBUG_TEXT
            std::cout << "\tTake-off: " << cur_item.frame << std::endl;
            std::cout << "\tTake-off - left?: " << cur_item.step_l << std::endl;
#endif
        //end if step_out_detected
        } else { 
#ifdef SHOW_DEBUG_TEXT
            std::cout << "\tNo step out found. " << std::endl;
#endif

            break; //No step out, means irrelevant rest of video
        }
        
        //Search for arrival info
        int last_arrival_frame = -1; //Value is -1 if no one steps, and the corresponding frame if it steps
        int nearest_arrival_code; //Stepping or not, it is the nearest arrival code
        int last_real = -1;
        float nearest_arrival_distance = FLT_MAX;
        bool still_near_center_l = true, still_near_center_r = true, real_arrival = false, is_left = true;
        for (int i = current_center_exit1<current_center_exit2?current_center_exit1:current_center_exit2; i < maxFrame; ++i) {
#ifdef SHOW_DEBUG_TEXT
            std::cout << "\tArrival - frame: " << i << std::endl;
#endif            
            //There might still be one of the feet near center
            if(ft.in_objective1[i] != 5) {
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tArrival - check left far center... " << std::endl;
#endif            
                still_near_center_l = false;
                if(ft.left_step[i] == 1) { 
#ifdef SHOW_DEBUG_TEXT
                    std::cout << "\tArrival - check left far center step - code: " << ft.in_objective1[i] << std::endl;
                    std::cout << "\tArrival - check left far center step - distance: " << ft.odist1[i] << std::endl;
                    std::cout << "\tArrival - check left far center step - last_real: " << last_real << std::endl;
#endif            
                    if(ft.odist1[i]==0 && last_real != ft.in_objective1[i]) { //Registers the first arrival with this code
                        last_arrival_frame = i;
                        last_real = nearest_arrival_code = ft.in_objective1[i];
                        nearest_arrival_distance = 0;
                        real_arrival = true;
                        is_left = true;
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tArrival - left real arrival found at: " << i << std::endl;
#endif            
                    } 
                    if(!real_arrival && ft.odist1[i] < nearest_arrival_distance) {
                        nearest_arrival_distance = ft.odist1[i];
                        last_arrival_frame = i;
                        nearest_arrival_code = ft.in_objective1[i];
                        is_left = true;
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tArrival - left near arrival found at: " << i << std::endl;
                std::cout << "\tArrival - left near arrival - nearest arrival distance: " << nearest_arrival_distance << std::endl;
#endif            
                    }
                }
            }
            if(ft.in_objective2[i] != 5) {
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tArrival - check right far center... " << std::endl;
#endif            
                still_near_center_r = false;
                if(ft.right_step[i] == 1) { 
#ifdef SHOW_DEBUG_TEXT
                    std::cout << "\tArrival - check right far center step - code: " << ft.in_objective2[i] << std::endl;
                    std::cout << "\tArrival - check right far center step - distance: " << ft.odist2[i] << std::endl;
                    std::cout << "\tArrival - check right far center step - last_real: " << last_real << std::endl;
#endif            
                    if(ft.odist2[i]==0 && last_real != ft.in_objective2[i]) { //Registers the first arrival with this code
                        last_arrival_frame = i;
                        last_real = nearest_arrival_code = ft.in_objective2[i];
                        nearest_arrival_distance = 0;
                        real_arrival = true;
                        is_left = false;
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tArrival - right real arrival found at: " << i << std::endl;
#endif            
                    } 
                    if(!real_arrival && ft.odist2[i] < nearest_arrival_distance) {
                        nearest_arrival_distance = ft.odist2[i];
                        last_arrival_frame = i;
                        nearest_arrival_code = ft.in_objective2[i];
                        is_left = false;
#ifdef SHOW_DEBUG_TEXT
                        std::cout << "\tArrival - right near arrival found at: " << i << std::endl;
                        std::cout << "\tArrival - right near arrival - nearest arrival distance: " << nearest_arrival_distance << std::endl;
#endif            
                    }
                }

            }
            
            if(!still_near_center_l && ft.in_objective1[i] == 5 && ft.left_step[i] == 1) { //First step returning to center
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tArrival - left step returning to center found at: " << i << std::endl;
#endif            

                next_seq = i;
                break;
            }
            if(!still_near_center_r && ft.in_objective2[i] == 5 && ft.right_step[i] == 1) { //First step returning to center
#ifdef SHOW_DEBUG_TEXT
                std::cout << "\tArrival - right step returning to center found at: " << i << std::endl;
#endif            
                next_seq = i;
                break;
            }
            
        } 
        
        //Set arrival errors and arrival time
        if(!real_arrival || nearest_arrival_code != cur_objective) {
            if(real_arrival && nearest_arrival_code != cur_objective)
                right_objective = false;
            if(!real_arrival)
                step_objective = false; 
        }
        cur_item.code = nearest_arrival_code;
        cur_item.d_l = ft.odist1[last_arrival_frame];
        cur_item.d_r = ft.odist2[last_arrival_frame];
        cur_item.step_l = is_left? 1:0;
        cur_item.step_r = is_left? 0:1;
        cur_item.frame = last_arrival_frame;
        cur_section.items.push_back(cur_item);
        cur_section.arrival_frame = last_arrival_frame;
        cur_section.arrival_code = nearest_arrival_code;

#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tArrival - arrival frame: " << cur_section.arrival_frame << std::endl;
        std::cout << "\tArrival - arrival code: " << cur_section.arrival_code << std::endl;
        if(is_left)
            std::cout << "\tArrival - arrival distance: " << ft.odist1[last_arrival_frame] << std::endl;
        else
            std::cout << "\tArrival - arrival distance: " << ft.odist2[last_arrival_frame] << std::endl;
#endif            

        
        //Check if steps in center on return
        bool still_far_center_l = true, still_far_center_r = true, ready_l = false, ready_r = false;
        step_center = false;
        for (int i = next_seq; i < maxFrame; ++i) {
#ifdef SHOW_DEBUG_TEXT
            std::cout << "\tReturn to center - frame: " << i << std::endl;
#endif            
            //There might still be one of the feet near center
            if(ft.in_objective1[i] == 5) {
                still_far_center_l = false;
                if(ft.left_step[i] == 1) {
#ifdef SHOW_DEBUG_TEXT
            std::cout << "\tReturn to center - found left at: " << i << std::endl;
#endif            
                    step_center = true;                    
                    break;
                }
            }
            if(ft.in_objective2[i] == 5) {
                still_far_center_r = false;
                if(ft.right_step[i] == 1) {
#ifdef SHOW_DEBUG_TEXT
                    std::cout << "\tReturn to center - found right at: " << i << std::endl;
#endif            
                    step_center = true;
                    break;
                }
            }
            
            if(!still_far_center_l && ft.in_objective1[i] != 5)
                ready_l = true;
                
            if(!still_far_center_r && ft.in_objective2[i] != 5)
                ready_r = true;
            
            if(ready_l && ready_r) {
#ifdef SHOW_DEBUG_TEXT
            std::cout << "\tReturn to center - both ready at: " << i << std::endl;
#endif            
                break;
            }
        } 
        
        //Set error and store in sequences
        cur_section.error = (right_objective && step_objective && step_center) ? false : true;
#ifdef SHOW_DEBUG_TEXT
        std::cout << "\tFinal stimulus error: " << cur_section.error << std::endl;
#endif            

        sequences.push_back(cur_section);
        
        //Set next beggining to next sequence of centers
        current_seq = next_seq;
        first_stimulus = false;
    } //end stimuli sequence
    
    return toJSON(sequences);
}

# toJSON

In [ ]:
std::string toJSON(const std::vector<Section>& sections) {
    std::string json = "[\n";
    for(size_t i = 0; i < sections.size(); ++i) {
        const Section& sec = sections[i];
        json += "    {\n";
        json += "    \"id_sequence\": " + std::to_string(i) + ",\n";
        json += "    \"takeoff_frame\": " + std::to_string(sec.takeoff_frame) + ",\n";
        json += "    \"arrival_frame\": " + std::to_string(sec.arrival_frame) + ",\n";
        json += "    \"error\": " + std::string(sec.error ? "true" : "false") + "\n";
        json += "    }";
        if (i < sections.size() - 1) json += ",";
        json += "\n";
    }
    json += "]";

    std::string json2 = "[\n";
    for(size_t i = 0; i < sections.size(); ++i) {
        const Section& sec = sections[i];
        json2 += "    {\n";
        json2 += "    \"id_sequence\": " + std::to_string(i) + ",\n";
        json2 += "    \"takeoff_frame\": " + std::to_string(sec.takeoff_frame) + ",\n";
        json2 += "    \"arrival_frame\": " + std::to_string(sec.arrival_frame) + ",\n";
        json2 += "    \"error\": " + std::string(sec.error ? "true" : "false") + ",\n";
        json2 += "    \"items\": [\n";
        for (size_t j = 0; j < sec.items.size(); ++j) {
            const item& it = sec.items[j];
            json2 += "        {\n";
            json2 += "        \"code\": " + std::to_string(it.code) + ",\n";
            json2 += "        \"intersects\": " + std::to_string(it.intersects) + ",\n";
            json2 += "        \"frame\": " + std::to_string(it.frame) + ",\n";
            json2 += "        \"d_l\": " + std::to_string(it.d_l) + ",\n";
            json2 += "        \"d_r\": " + std::to_string(it.d_r) + ",\n";
            json2 += "        \"step_l\": " + std::string(it.step_l ? "true" : "false") + ",\n";
            json2 += "        \"step_r\": " + std::string(it.step_r ? "true" : "false") + "\n";
            json2 += "        }";
            if (j < sec.items.size() - 1) json2 += ",";
            json2 += "\n";
        }
        json2 += "    ]\n";
        json2 += "    }";
        if (i < sections.size() - 1) json2 += ",";
        json2 += "\n";
    }
    json2 += "]";

    std::cout << "JSON NUEVO \n\n" << json2 << std::endl;

    return json;
}